# RETAIL DATASET ANALYSIS <br>
<b>Author: CUONG VO <br>
Student ID: 131116 </b>

Technical Information: 
<b>
- OS: WSL2 Ubuntu 22.04LTS
- Spark: 3.5.0
- Scala: 2.12.18
- Java OpenJDK: 11.0.21 
</b>

### Library Import

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

### Create Spark Session

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Retail Analysis').getOrCreate()
sc = spark.sparkContext

### Dataset import

<b>Source:</b> https://archive.ics.uci.edu/dataset/502/online+retail+ii

<b>Abstract:</b> This dataset captures essential details from online retail transactions, encompassing diverse aspects of the purchase process. This comprehensive dataset allows for the analysis of purchasing patterns, sales trends, and customer behavior across different countries. It provides a valuable resource for exploring various aspects of online retail operations, from product popularity to geographical sales distribution. The inclusion of detailed transaction information enables researchers and analysts to delve into the dynamics of the online retail business and derive insights that can inform strategic decisions.

In [4]:
retail_schema = StructType([
  StructField('Invoice', StringType()),
  StructField('StockCode', StringType()),
  StructField('Description', StringType()),
  StructField('Quantity', IntegerType()),
  StructField('InvoiceDate', StringType()),
  StructField('Price', FloatType()),
  StructField('Customer ID', StringType()),
  StructField('Country', StringType())
])

In [5]:
file_location = "./online_retail.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(retail_schema) \
  .load(file_location)

df.limit(10).show()


+-------+---------+--------------------+--------+-------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|  InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+-------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|01-12-09 7:45| 6.95|    13085.0|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|01-12-09 7:45| 6.75|    13085.0|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|01-12-09 7:45| 6.75|    13085.0|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|01-12-09 7:45|  2.1|    13085.0|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|01-12-09 7:45| 1.25|    13085.0|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|01-12-09 7:45| 1.65|    13085.0|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|01-12-09 7:45| 1.25|    13085.0|United Kingdom|
| 489434|    21523|FANCY FONT 

In [6]:
df.createOrReplaceTempView("Retail_data")

### Descriptive Analysis

In [7]:
from pyspark.sql.functions import *

df = df.withColumn("Sales", round(df.Quantity * df.Price,2) )
df_customer = df.groupBy('Customer ID').agg(round(sum("Sales"),2).alias("Total_Sales"))
df_customer.limit(10).show()

+-------+-----------+
|Invoice|Total_Sales|
+-------+-----------+
| 489677|      192.0|
|C491017|      -4.95|
| 491045|      303.2|
| 491658|     155.06|
|C491705|      -22.5|
|C492541|      -99.0|
|C493168|     -177.6|
| 493542|     118.75|
| 493977|     275.95|
|C493984|     -10.43|
+-------+-----------+



In [8]:
df_product = df.groupBy('StockCode').agg(
    round(sum("Sales"),2).alias("Total_Sales")
df_product.limit(10).show()

+---------+-----------+--------------+---------+
|StockCode|Total_Sales|Total_Quantity|Avg_Price|
+---------+-----------+--------------+---------+
|    21248|    1276.54|           938|     1.36|
|    22121|    9488.68|          1484|     6.39|
|    21889|   17442.57|         13077|     1.33|
|    22254|    1718.46|          1339|     1.28|
|   84899F|     192.17|            67|     2.87|
|    21249|    4815.64|          1670|     2.88|
|    21259|   14408.43|          2335|     6.17|
|   90197B|     322.52|            64|     5.04|
|    21894|    1886.22|          1312|     1.44|
|    90022|      136.1|            36|     3.78|
+---------+-----------+--------------+---------+



In [11]:
df_temp = df_customer.toPandas()
df_temp.isna().sum()

Invoice        0
Total_Sales    0
dtype: int64

In [10]:
df_temp = df_product.toPandas()
df_temp.isna().sum()

StockCode          0
Total_Sales        0
Total_Quantity     0
Avg_Price         19
dtype: int64

In [12]:
df_product.dropna().count()

5286

In [ ]:
df_customer.dropna().count()

### Insight